In [46]:
import pandas as pd
import math

In [4]:
df = pd.read_csv("data/game_data_public.VOW.PremierDraft.csv")

c:\users\vitoj\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.shape

(494030, 1106)

In [42]:
average_wp = df.won.sum() / len(df)

In [15]:
drawn_cards = df.columns[["drawn_" in name for name in df.columns]]

In [21]:
relevant = df[(df.drawn_Abrade>0) | ((df.opening_hand_Abrade>0))]
relevant.won.sum() / len(relevant)

0.5808695652173913

In [52]:
def assign_grade(wp):
    if abs(wp-average_wp) < 0.025: 
        return "C"
    if wp > average_wp + 0.075:
        return "A"
    if wp > average_wp:
        return "B"
    if wp < average_wp -0.075:
        return "F"
    if wp < average_wp:
        return "D"

In [53]:
win_rates = []
for name in drawn_cards:
    base_name = name.split("_")[-1]
    df["times_in_hand"] = df["drawn_"+base_name] + df["opening_hand_"+base_name]
    won_games = (df["times_in_hand"] * df.won).sum()
    all_games = df["times_in_hand"].sum()
    win_rate = won_games / all_games
    win_rates.append((base_name, assign_grade(win_rate), win_rate))

In [58]:
data = pd.DataFrame(win_rates, columns = ["name", "grade", "win_rate"])

In [83]:
%run "card_loader.ipynb"

In [85]:
data["rarity"] = data["name"].apply(lambda x: cards[cards.name_short == x].iloc[0].rarity)

In [95]:
data["colors"] = data["name"].apply(lambda x: cards[cards.name_short == x].iloc[0].colors)

In [96]:
data.to_csv("./results/card_scores_vows.csv")

In [90]:
for letter in ["A", "B", "C", "D", "F"]:
    string = letter 
    for rarity in ["mythic", "rare", "uncommon", "common"]:
        string += " "+str(sum((data["grade"]==letter) & (data["rarity"]==rarity)))
    print (string)
    
    #print (letter, sum(data["grade"]==letter) / len(data)) 

A 6 7 1 0
B 7 17 18 4
C 6 26 36 68
D 0 9 23 31
F 1 6 4 2


In [92]:
data[(data["grade"]=="C") & (data["rarity"]=="mythic")]

,name,grade,win_rate,rarity
38,Cemetery Gatekeeper,C,0.554118,mythic
41,Cemetery Prowler,C,0.562850,mythic
43,"Chandra, Dressed to Kill",C,0.530933,mythic
58,Cultivator Colossus,C,0.538260,mythic
114,Hallowed Haunting,C,0.560033,mythic
201,Savior of Ollenbock,C,0.565003,mythic


In [ ]:
data.to

In [54]:
sorted(win_rates, key=lambda x: -x[2])

[('Toxrill, the Corrosive', 'A', 0.6751697559185171),
 ('Halana and Alena, Partners', 'A', 0.6720264317180616),
 ('Wedding Announcement', 'A', 0.6668845315904139),
 ('Dreadfeast Demon', 'A', 0.6646982838162022),
 ('Avabruck Caretaker', 'A', 0.6606929510155317),
 ('Henrika Domnathi', 'A', 0.6598257502420135),
 ('Katilda, Dawnhart Martyr', 'A', 0.6489242631513493),
 ('Cemetery Desecrator', 'A', 0.6398505114083399),
 ('Volatile Arsonist', 'A', 0.6358595194085028),
 ('Anje, Maid of Dishonor', 'A', 0.6348399653979239),
 ('Sorin the Mirthless', 'A', 0.6325036603221084),
 ('Dreamshackle Geist', 'A', 0.626878488621726),
 ('Welcoming Vampire', 'A', 0.6253035734936973),
 ('Bloodtithe Harvester', 'A', 0.6225484942664238),
 ('Hullbreaker Horror', 'B', 0.6213887480993411),
 ('Bloodvial Purveyor', 'B', 0.617231207405147),
 ('Voldaren Bloodcaster', 'B', 0.6139846743295019),
 ('Faithbound Judge', 'B', 0.6119041858214372),
 ('Stensia Uprising', 'B', 0.6115942028985507),
 ('Edgar, Charmed Groom', 'B', 0

In [10]:
pd.Series(df.columns).apply(lambda x: x.split("_")[0] ).unique()

array(['user', 'draft', 'build', 'expansion', 'event', 'game', 'rank',
       'opp', 'main', 'splash', 'on', 'num', 'won', 'opening', 'drawn',
       'deck', 'sideboard'], dtype=object)